# Loading Data from Assignment-1

In [21]:
import glob
import os
import xml.etree.ElementTree as ET

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, silhouette_score, adjusted_rand_score, \
    homogeneity_score, completeness_score, adjusted_mutual_info_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from keras.layers import Input, Dense
from keras.models import Model
import tensorflow as tf

DataFrameColumns = ['Headline', 'Text', 'Bip_Topics', 'Date_Published', 'Item_ID', 'File_Name']
biptopiclist = []
rows = []
DataFrameCustom = []
stopwords = set(stopwords.words('english'))
lists = []


def dataextraction():
    for file in glob.iglob('/Users/santoshkumarmedisetty/Downloads/testing/*/*.xml'):
        bip_topic = []
        mytree = ET.parse(file)
        myroot = mytree.getroot()
        path, filename = os.path.split(file)
        itemid = myroot.attrib.get('itemid')
        hl = myroot.find('headline').text
        dc = myroot.findall("./metadata/dc[@element='dc.date.published']")
        datepublished = dc[0].attrib.get("value")
        paragraph = ""
        textnode = myroot.find("text")
        for node in textnode:
            paragraph = paragraph + node.text

        LemmatizedSentence = stemlemm(paragraph)

        Sentencewostopwords = stopword(LemmatizedSentence)
        Sentencewostopwords = ' '.join(Sentencewostopwords)
        cd = myroot.findall("./metadata/codes[@class='bip:topics:1.0']/code")

        for i in cd:
            k = i.attrib.get("code")
            biptopiclist.append(k)
            rows.append(
                {"Headline": hl, "Text": Sentencewostopwords, "Bip_Topics": k, "Date_Published": datepublished,
                 "Item_ID": itemid,
                 "File_Name": filename})
            break

    DataFrameCustom = pd.DataFrame(rows, columns=DataFrameColumns)
    DataFrameCustom.to_csv('/Users/santoshkumarmedisetty/Downloads/ML assignment/FirstDF.csv')
    return DataFrameCustom

def stopword(paragraph):
    words_tokens = word_tokenize(paragraph)
    sentence = []
    for w in words_tokens:
        if w not in stopwords:
            sentence.append(w)
    return sentence


def uniquebipcodes(biptopiclist):
    unique_codes = []
    for code in biptopiclist:
        if code not in unique_codes:
            unique_codes.append(code)
    print(unique_codes)
    return unique_codes


def stemlemm(sentence):
    words_tokens = word_tokenize(sentence)
    filtered_sentence_list = [w for w in words_tokens if w not in stopwords]

    StemSentence = []
    LemmSentence = []
    Stemmed_Sentence = []
    for word in filtered_sentence_list:
        StemSentence.append(PorterStemmer().stem(word))

    for wor in StemSentence:
        LemmSentence.append(WordNetLemmatizer().lemmatize(wor))

    Stemmed_Sentence = " ".join(StemSentence)
    Lemmatized_Sentence = " ".join(LemmSentence)

    return Lemmatized_Sentence

FirstDF=dataextraction()

In [22]:
FirstDF

,Headline,Text,Bip_Topics,Date_Published,Item_ID,File_Name
0,Care Group Inc Q4 shr loss vs profit.,( 000 's omit ) year end decemb 31 ( audit ) 1...,C15,1997-03-31,476242,476242newsML.xml
1,China economy to grow at more than 10 pct in 1...,"china see econom growth 10 percent year , infl...",E11,1997-03-31,476768,476768newsML.xml
2,Africans say global debt relief plan too rigid.,african financi leader criticis monday new int...,E51,1997-03-31,476074,476074newsML.xml
3,Argentine export grain prices - March 31.,"export offer/bid price u.s. dollar per tonn , ...",M14,1997-03-31,476597,476597newsML.xml
4,Yemen says kidnappers not harming German hosta...,four german tourist kidnap mountain tribesman ...,GCAT,1997-03-31,476845,476845newsML.xml
...,...,...,...,...,...,...
2799,California ordered to hire welfare recipients.,california governor pete wilson friday order s...,E41,1997-03-29,475206,475206newsML.xml
2800,Dutch blaze victims buried in eastern Turkey.,bodi turkish woman five child die suspect raci...,GCAT,1997-03-29,475340,475340newsML.xml
2801,Court paves way for Oklahoma trial to start.,A feder appeal court friday deni three request...,GCAT,1997-03-29,475389,475389newsML.xml
2802,NBA BASKETBALL-STANDINGS AFTER FRIDAY'S GAMES.,stand nationalbasketbal associ team game play ...,GCAT,1997-03-29,475176,475176newsML.xml


# Creating Clusters and assigning Cluster IDs

In [23]:
def clustering(DataFrameCustom):

    textdata = DataFrameCustom["Text"]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(textdata)
    n_clusters = 10

    model = KMeans(n_clusters)

    model.fit(X)
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    
    print('Top 10 words in each cluster are')
    for i in range(n_clusters):
        print('Cluster % d:' % i)
        for ind in order_centroids[i, :10]:
            print(' % s' % terms[ind])


    results = pd.DataFrame()
    results['FileName'] = DataFrameCustom["File_Name"]
    results['text'] = DataFrameCustom["Text"]
    clusters = model.labels_

    results['Cluster'] = clusters
    results['Bip_Topics'] = DataFrameCustom["Bip_Topics"]
    results.to_csv('/Users/santoshkumarmedisetty/Downloads/ML assignment/clusteredDF.csv')
    
clusteredDF=clustering(FirstDF)

Top 10 words in each cluster are
Cluster  0:
 cup
 match
 game
 saturday
 second
 play
 world
 beat
 team
 indi
Cluster  1:
 percent
 said
 market
 rate
 stock
 trade
 price
 trader
 dealer
 close
Cluster  2:
 compani
 said
 share
 million
 inc
 stock
 corp
 quarter
 monday
 1996
Cluster  3:
 loss
 net
 incom
 share
 1996
 shr
 31
 profit
 per
 000
Cluster  4:
 congress
 parti
 gowda
 front
 deve
 india
 said
 govern
 kashmir
 support
Cluster  5:
 00
 50
 01
 97
 pct
 10
 15
 amount
 70
 03
Cluster  6:
 arab
 israel
 palestinian
 isra
 peac
 netanyahu
 jerusalem
 minist
 settlement
 arafat
Cluster  7:
 said
 th
 state
 tonn
 china
 new
 would
 monday
 year
 govern
Cluster  8:
 bank
 billion
 said
 rate
 million
 bond
 percent
 year
 central
 issu
Cluster  9:
 said
 polic
 rebel
 peopl
 forc
 th
 albanian
 refuge
 albania
 govern


# Applying different Classifier to each cluster

In [24]:
df=pd.read_csv('/Users/santoshkumarmedisetty/Downloads/ML assignment/clusteredDF.csv')
#     performanceevaluation(results,X)
n_clusters=10
for i in range(n_clusters):
    lists.append(df.loc[df["Cluster"] == i])
    # print(lists)
    # print(type(lists[4]))

def featureextraction(FirstDF):
    vectorizer = CountVectorizer()
    textdata = FirstDF["text"]
    bip = FirstDF["Bip_Topics"]
    biparray = pd.Series(bip).values
    filename = FirstDF["FileName"]
    FeatureData = vectorizer.fit_transform(textdata).toarray()
    FeatureDataFrameData = np.column_stack((FeatureData, biparray))
    dataFrameColumns = vectorizer.get_feature_names()
    dataFrameColumns.insert(len(dataFrameColumns), 'Bip_Topics')
    FeatureDataFrame = pd.DataFrame(data=FeatureDataFrameData, columns=dataFrameColumns, index=filename)
    FeatureDataFrame.to_csv('/Users/santoshkumarmedisetty/Downloads/ML assignment/FeaturedataDF.csv')
    return FeatureDataFrame


def datasplitter(DF,clusternumber):

    X = DF.iloc[:, :-1].values

    y = DF["Bip_Topics"]


        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    # PCA
    pca = PCA(n_components=10)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)


    if(clusternumber==1):
        print("KNN is applied for cluster number 1")
        # KNN
        KNN_model = KNeighborsClassifier(n_neighbors=10)
        KNN_model.fit(X_train, y_train)
        KNN_prediction = KNN_model.predict(X_test)
        print("Metrics of KNN")
        qualityofclassifier(KNN_prediction, y_test)

    if (clusternumber == 2):
        print("SVM is applied for cluster number 2")
        # #SVM
        SVC_model = svm.SVC(kernel='linear', C=0.1)
        SVC_model.fit(X_train, y_train)
        SVC_prediction = SVC_model.predict(X_test)
        print("Metrics of SVM are")
        qualityofclassifier(SVC_prediction, y_test)

    if (clusternumber == 0):
        print("Random Forest is applied for cluster number 0")
        # RandomForest
        RF = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=0)
        RF.fit(X_train, y_train)
        RF_Prediction = RF.predict(X_test)
        print("Metrics of Random Forest are")
        qualityofclassifier(RF_Prediction, y_test)

    if (clusternumber == 4):
        print("Decision Tree Classifier is applied for cluster number 4")
        # DecisionTreeClassifier
        DT_gini = DecisionTreeClassifier(criterion="gini", random_state=100, max_depth=3, min_samples_leaf=10)
        DT_gini.fit(X_train, y_train)
        DT_pred = DT_gini.predict(X_test)
        print("Metrics of DT are")
        qualityofclassifier(DT_pred, y_test)

    if (clusternumber == 6):
        print("Neural Network Classifier is applied for cluster number 6")
        # NeuralNetwork
        p = MLPClassifier(random_state=42, tol=0.001, max_iter=500)
        p.fit(X_train, y_train)
        NN_Pred = p.predict(X_test)
        print("Metrics of NN are")
        qualityofclassifier(NN_Pred, y_test)

    if (clusternumber == 3):
        print("Naive Bayes Classifier is applied for cluster number 3")
        # Naive Bayes
        GNB = GaussianNB()
        GNB.fit(X_train, y_train)
        GNB_Pred = GNB.predict(X_test)
        print("Metrics of Naive Bayes are")
        qualityofclassifier(GNB_Pred, y_test)

    if(clusternumber==5):
        print("Stochastic Gradient Descent Classifier is applied for cluster number 5")
        #Stochastic Gradient Descent
        SGD=SGDClassifier()
        SGD.fit(X_train,y_train)
        SGD_Pred=SGD.predict(X_test)
        print("Metrics of SGD are")
        qualityofclassifier(SGD_Pred, y_test)

    if (clusternumber == 8):
        print("Gaussian Process  Classifier is applied for cluster number 8")
        # Gaussian Process  Classifier
        GPC = GaussianProcessClassifier()
        GPC.fit(X_train, y_train)
        GPC_Pred = GPC.predict(X_test)
        print("Metrics of GPC are")
        qualityofclassifier(GPC_Pred, y_test)

    if (clusternumber == 7):
        print("Adaboost  Classifier is applied for cluster number 7")
        # Adaboost  Classifier
        ADA = AdaBoostClassifier()
        ADA.fit(X_train, y_train)
        ADA_Pred = ADA.predict(X_test)
        print("Metrics of Adaboost are")
        qualityofclassifier(ADA_Pred, y_test)

    if (clusternumber == 9):
        print("Gradient Boosting  Classifier is applied for cluster number 9")
        # Gradient Boosting  Classifier
        GB = GradientBoostingClassifier()
        GB.fit(X_train, y_train)
        GB_Pred = GB.predict(X_test)
        print("Metrics of Gradient Boosting are")
        qualityofclassifier(GB_Pred, y_test)





def qualityofclassifier(prediction, testlabel):
        print("Confusion Matrix :" + str(confusion_matrix(prediction, testlabel)))
        print("Classification Report: " + str(classification_report(prediction, testlabel)))
        print("Accuracy is: " + str(accuracy_score(prediction, testlabel)))


for j in range(n_clusters):
    datasplitter(featureextraction(lists[j]),j)



Random Forest is applied for cluster number 0
Metrics of Random Forest are
Confusion Matrix :[[83]]
Classification Report:               precision    recall  f1-score   support

        GCAT       1.00      1.00      1.00        83

    accuracy                           1.00        83
   macro avg       1.00      1.00      1.00        83
weighted avg       1.00      1.00      1.00        83

Accuracy is: 1.0
KNN is applied for cluster number 1
Metrics of KNN
Confusion Matrix :[[ 1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  1  1  0  1  0  0  0]
 [ 1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 

/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


SVM is applied for cluster number 2
Metrics of SVM are
Confusion Matrix :[[ 3  0  3  0  2  0  0  1  2  0  0  0  0  0  0  1  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  1  4  4 34  0  1  9 13  1  0  2  3  1  7  1  2  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  1  0  1  0  0  0  0  0  0  0]
 [ 2  1  1  0  2  0  0  0  4  0  0  0  1  0  2  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  

/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Naive Bayes Classifier is applied for cluster number 3
Metrics of Naive Bayes are
Confusion Matrix :[[60]]
Classification Report:               precision    recall  f1-score   support

         C15       1.00      1.00      1.00        60

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60

Accuracy is: 1.0
Decision Tree Classifier is applied for cluster number 4
Metrics of DT are
Confusion Matrix :[[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 1  1 22  1  3]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
Classification Report:               precision    recall  f1-score   support

         C31       0.00      0.00      0.00         0
         E51       0.00      0.00      0.00         0
        GCAT       1.00      0.79      0.88        28
        GPOL       0.00      0.00      0.00         0
         M11       0.00      0.00      0.00         0

    accuracy                           0.79    

...

/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Gaussian Process  Classifier is applied for cluster number 8
Metrics of GPC are
Confusion Matrix :[[1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 8 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 2 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [2 0 0 1 0 0 0 2 0 2 3 0 0 0 0 1]
 [0 0 0 0 0 2 0 0 0 0 5 0 0 0 0 2]
 [1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 4 1 0 0 1 1 3 2 0 1 0 5]]
Classification Report:               precision    recall  f1-score   support

         C11       0.14      0.25      0.18         4
         C12       0.00      0.00      0.00         0
         C13       0.33      0.20      0.25         5
         C14       0.00      0.00      0.00         0
         C15       0.57      0.73      0.64    

/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Gradient Boosting  Classifier is applied for cluster number 9
Metrics of Gradient Boosting are
Confusion Matrix :[[ 0  0  0  1]
 [ 0  0  0  0]
 [ 0  0  0  0]
 [ 0  2  8 76]]
Classification Report:               precision    recall  f1-score   support

         C24       0.00      0.00      0.00         1
        CCAT       0.00      0.00      0.00         0
         E51       0.00      0.00      0.00         0
        GCAT       0.99      0.88      0.93        86

    accuracy                           0.87        87
   macro avg       0.25      0.22      0.23        87
weighted avg       0.98      0.87      0.92        87

Accuracy is: 0.8735632183908046


/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# Evaluating the performance of each cluster

In [25]:
results=pd.read_csv('/Users/santoshkumarmedisetty/Downloads/ML assignment/clusteredDF.csv')
X = TfidfVectorizer().fit_transform(FirstDF["Text"])
def performanceevaluation(clusteredDF,fitdata):
    
    print("Completeness score" + str(completeness_score(clusteredDF["Bip_Topics"], clusteredDF["Cluster"])));
    print("Homogenity score is" + str(homogeneity_score(clusteredDF["Bip_Topics"], clusteredDF["Cluster"])));
    print("Silhoutte score is" + str(silhouette_score(fitdata, clusteredDF["Cluster"], metric='euclidean')));
    print("Rand Score is" + str(adjusted_rand_score(clusteredDF["Bip_Topics"], clusteredDF["Cluster"])));
    print("Mutual Info Score is" + str(adjusted_mutual_info_score(clusteredDF["Bip_Topics"], clusteredDF["Cluster"])));
    
performanceevaluation(results,X)


# Performance evaluation:
# In general, Silhoutte score is the best parameter to evaluate the performance of the cluster.

# Silhouette score : 
# It evaluates the consistency of data within the cluster by calculating the distance between each object of a cluster to other objects of the same cluster as well as the objects from the neighboring clusters.
# Unlike other evaluation techniques that require true labels of a dataset, Silhouette works only with data and clustering results.
# The score is the measure of how similar an object is in its cluster when compared to an object in another cluster.
# It ranges from -1 to +1, where the value near +1 indicates that the object is well-matched with the own cluster and doesn't belong to the neighboring cluster and the vale near -1 indicates that the assignment of that object to the cluster should be changed(i.e. the object doesn't belong to that cluster).
# If the silhouette score is 0, then the object is in between the two clusters.

# adjusted rand score:
# Rand index computes the similarity between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings.
# The adjusted rand index is the modified version of the rand index which finds similarity of the cluster assignments of the objects ignoring permutations and normalizations.
# Its boundary ranges from -1 to +1 where negative values are bad and +1 is a perfect match score.
# It is symmetric (i.e. swapping the argument doesn't change the score)
# It doesn't depend on cluster structures (i.e. two clusters with varying structure can be compared)


Completeness score0.39439488090209435
Homogenity score is0.32712930655632644
Silhoutte score is0.014990561527312865
Rand Score is0.15047623103002636
Mutual Info Score is0.30970460032500696


/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


# Applying AutoEncoder to enhance the features

In [30]:
def clustering(DataFrameCustom):
    textdata = DataFrameCustom["Text"]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(textdata)
    n_clusters = 10

    model = KMeans(n_clusters)

    model.fit(X)
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()


    for i in range(n_clusters):
        top_words = [terms[ind] for ind in order_centroids[i, :20]]
    print('\n')

    results = pd.DataFrame()
    results['FileName'] = DataFrameCustom["File_Name"]
    results['text'] = DataFrameCustom["Text"]
    clusters = model.labels_

    results['Cluster'] = clusters
    results['Bip_Topics'] = DataFrameCustom["Bip_Topics"]
    results.to_csv('/Users/santoshkumarmedisetty/Downloads/ML assignment/clusteredDF.csv')
    df=pd.read_csv('/Users/santoshkumarmedisetty/Downloads/ML assignment/clusteredDF.csv')
#     performanceevaluation(results,X)
    for i in range(n_clusters):
        lists.append(df.loc[df["Cluster"] == i])


    def featureextraction(FirstDF):
        vectorizer = CountVectorizer()
        textdata = FirstDF["text"]
        bip = FirstDF["Bip_Topics"]
        biparray = pd.Series(bip).values
        filename = FirstDF["FileName"]
        FeatureData = vectorizer.fit_transform(textdata).toarray()
        FeatureDataFrameData = np.column_stack((FeatureData, biparray))
        dataFrameColumns = vectorizer.get_feature_names()
        dataFrameColumns.insert(len(dataFrameColumns), 'Bip_Topics')
        FeatureDataFrame = pd.DataFrame(data=FeatureDataFrameData, columns=dataFrameColumns, index=filename)
        FeatureDataFrame.to_csv('/Users/santoshkumarmedisetty/Downloads/ML assignment/FeaturedataDF.csv')
        return FeatureDataFrame


    def datasplitter(DF,clusternumber):

        X = DF.iloc[:, :-1].values

        y = DF["Bip_Topics"]

        input_layer = Input(shape=(X.shape[1],))
        encoded = Dense(50, activation='relu')(input_layer)
        decoded = Dense(X.shape[1], activation='softmax')(encoded)
        autoencoder = Model(input_layer, decoded)
        autoencoder.compile(optimizer='adam', loss='binary_crossentropy')


        X_train, X_test, y_train, y_test=train_test_split(X, X, test_size=0.3, random_state=101)

        global X_ae
        autoencoder.fit(X_train, y_train,
                        epochs=50,
                        batch_size=100,
                        shuffle=True,
                        verbose=30,
                        validation_data=(X_test, y_test))
        encoder = Model(input_layer, encoded)
        X_ae = encoder.predict(X)
        


        X_train, X_test, y_train, y_test = train_test_split(X_ae, y, test_size=0.3, random_state=101)
        print('Metrics after enhancement:')

        if(clusternumber==1):
            print("KNN is applied for cluster number 1")
            # KNN
            KNN_model = KNeighborsClassifier(n_neighbors=10)
            KNN_model.fit(X_train, y_train)
            KNN_prediction = KNN_model.predict(X_test)
            print("Metrics of KNN")
            qualityofclassifier(KNN_prediction, y_test)

        if (clusternumber == 2):
            print("SVM is applied for cluster number 2")
            # #SVM
            SVC_model = svm.SVC(kernel='linear', C=0.1)
            SVC_model.fit(X_train, y_train)
            SVC_prediction = SVC_model.predict(X_test)
            print("Metrics of SVM are")
            qualityofclassifier(SVC_prediction, y_test)

        if (clusternumber == 0):
            print("Random Forest is applied for cluster number 0")
            # RandomForest
            RF = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=0)
            RF.fit(X_train, y_train)
            RF_Prediction = RF.predict(X_test)
            print("Metrics of Random Forest are")
            qualityofclassifier(RF_Prediction, y_test)

        if (clusternumber == 4):
            print("Decision Tree Classifier is applied for cluster number 4")
            # DecisionTreeClassifier
            DT_gini = DecisionTreeClassifier(criterion="gini", random_state=100, max_depth=3, min_samples_leaf=10)
            DT_gini.fit(X_train, y_train)
            DT_pred = DT_gini.predict(X_test)
            print("Metrics of DT are")
            qualityofclassifier(DT_pred, y_test)

        if (clusternumber == 6):
            print("Neural Network Classifier is applied for cluster number 6")
            # NeuralNetwork
            p = MLPClassifier(random_state=42, tol=0.001, max_iter=500)
            p.fit(X_train, y_train)
            NN_Pred = p.predict(X_test)
            print("Metrics of NN are")
            qualityofclassifier(NN_Pred, y_test)

        if (clusternumber == 3):
            print("Naive Bayes Classifier is applied for cluster number 3")
            # Naive Bayes
            GNB = GaussianNB()
            GNB.fit(X_train, y_train)
            GNB_Pred = GNB.predict(X_test)
            print("Metrics of Naive Bayes are")
            qualityofclassifier(GNB_Pred, y_test)

        if(clusternumber==5):
            print("Stochastic Gradient Descent Classifier is applied for cluster number 5")
            #Stochastic Gradient Descent
            SGD=SGDClassifier()
            SGD.fit(X_train,y_train)
            SGD_Pred=SGD.predict(X_test)
            print("Metrics of SGD are")
            qualityofclassifier(SGD_Pred, y_test)

        if (clusternumber == 8):
            print("Gaussian Process  Classifier is applied for cluster number 8")
            # Gaussian Process  Classifier
            GPC = GaussianProcessClassifier()
            GPC.fit(X_train, y_train)
            GPC_Pred = GPC.predict(X_test)
            print("Metrics of GPC are")
            qualityofclassifier(GPC_Pred, y_test)

        if (clusternumber == 7):
            print("Adaboost  Classifier is applied for cluster number 7")
            # Adaboost  Classifier
            ADA = AdaBoostClassifier()
            ADA.fit(X_train, y_train)
            ADA_Pred = ADA.predict(X_test)
            print("Metrics of Adaboost are")
            qualityofclassifier(ADA_Pred, y_test)

        if (clusternumber == 9):
            print("Gradient Boosting  Classifier is applied for cluster number 9")
            # Gradient Boosting  Classifier
            GB = GradientBoostingClassifier()
            GB.fit(X_train, y_train)
            GB_Pred = GB.predict(X_test)
            print("Metrics of Gradient Boosting are")
            qualityofclassifier(GB_Pred, y_test)



    def qualityofclassifier(prediction, testlabel):
            print("Confusion Matrix :" + str(confusion_matrix(prediction, testlabel)))
            print("Classification Report: " + str(classification_report(prediction, testlabel)))
            print("Accuracy is: " + str(accuracy_score(prediction, testlabel)))


    for j in range(n_clusters):
        datasplitter(featureextraction(lists[j]),j)
clustering(FirstDF)



# Autoencoder:
# An Autoencoder mentioned in paper-1 is implemented with 50 input nodes for the encoder is created and then this encoded output is then decoded to obtain enhanced features.

# Autoencoder is a feed-forward neural network with the goal of dimensionality reduction.
# The input and output layer has the same number of features whereas it varies for the hidden layer(i.e. it has fewer units than input and output layer).
# The hidden layer constitutes of two major parts called encoder that maps inputs to the code and decoder that converts the code to the original input.
# We used relu and softmax activation functions in encoder and decoder. An activation function calculates a weighted sum of inputs, adds bias to it for each element.
# we have employed adam optimizer and binary cross-entropy loss function during compilation.
# The training of autoencoder is the same as that of conventional neural networks with backpropagation except that the error is calculated by comparing the output with its input itself.



Train on 193 samples, validate on 83 samples
Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50
Metrics after enhancement:
Random Forest is applied for cluster number 0
Metrics of Random Forest are
Confusion Matrix :[[83]]
Classification Report:               precision    recall  f1-score   support

        GCAT       1.00      1.00      1.00        83

    accuracy                           1.00        83
   macro avg       1.00      1.00      1.00     

/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Train on 272 samples, validate on 117 samples
Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50
Metrics after enhancement:
SVM is applied for cluster number 2
Metrics of SVM are
Confusion Matrix :[[ 1  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 1  0  0  1  2  0  0  0  3  0  0  0  0  0  0  0  0  1]
 [ 4  0  2  2 36  0  1  3  7  1  1 

/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Train on 139 samples, validate on 60 samples
Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50
Metrics after enhancement:
Naive Bayes Classifier is applied for cluster number 3
Metrics of Naive Bayes are
Confusion Matrix :[[60]]
Classification Report:               precision    recall  f1-score   support

         C15       1.00      1.00      1.00        60

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00

...

/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Train on 172 samples, validate on 75 samples
Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50
Metrics after enhancement:
Gaussian Process  Classifier is applied for cluster number 8
Metrics of GPC are
Confusion Matrix :[[ 2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0 10  1  0 

/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Train on 203 samples, validate on 87 samples
Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50
Metrics after enhancement:
Gradient Boosting  Classifier is applied for cluster number 9
Metrics of Gradient Boosting are
Confusion Matrix :[[ 0  0  0]
 [ 0  0  0]
 [ 2  8 77]]
Classification Report:               precision    recall  f1-score   support

        CCAT       0.00      0.00      0.00         0
         E51       0.00      0.00      0.00         0
 

/Users/santoshkumarmedisetty/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras import Sequential
df2=pd.read_csv('/Users/santoshkumarmedisetty/Downloads/ML assignment/FeaturedataDF.csv')
y=df2["Bip_Topics"]


def deepneuralclassifier(X_ae,y):
    X_train, X_test, y_train, y_test = train_test_split(X_ae, y, test_size=0.3, random_state=101)
    labelencoder = LabelEncoder()
    y1 = labelencoder.fit_transform(y_train)
    cvb = np.unique(y1)
    si = cvb.size


    classifier = Sequential()
    # First Hidden Layer
    classifier.add(Dense(8, activation='relu', kernel_initializer='random_normal', input_dim=X_ae.shape[1]))
    # Second  Hidden Layer
    classifier.add(Dense(8, activation='relu', kernel_initializer='random_normal'))
    # Output Layer
    classifier.add(Dense(si, activation='sigmoid', kernel_initializer='random_normal'))
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    classifier.fit(X_train, y1)
    
deepneuralclassifier(X_ae,y)

# deep neural network:

# A Neural Network with 2 hidden layers is created and advanced features are given as input.
# Since we have large sets of data here, a deep neural network performs better than conventional machine learning classifiers as deep learning can model large data and perform complex computations.
# It doesn't depend solely on feature engineering.
# It doesn't require labels or structured data as each layer depends on the output processed by the previous layer.
# A conventional classifier learns the model and then use those features to produce outputs. However, if the output is not the desired one, the features extracted needs to be retained manually.
# Whereas deep neural networks did not require manual intervention as the nested layers retain the data which will aid in learning from their errors.